In [1]:
import numpy as np
import xgboost as xgb
from wideboost.wrappers import wxgb
from wideboost.wrappers.wxgb import wxgbModel

import tensorflow as tf
import tensorflow_datasets as tfds
from matplotlib import pyplot as plt

ds_train, ds_info = tfds.load(
    'titanic',
    split=['train'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

for i in ds_train[0].batch(60000):
    a = i

In [2]:
## Keep only numeric columns. Randomly split the dataset into training and test

X = []
for s in a[0].keys():
    if a[0][s].dtype != tf.string:
        X.append(a[0][s].numpy().reshape([-1,1]))

X = np.concatenate(X,axis=1)

idx = np.arange(X.shape[0])
np.random.seed(1)
train_idx = np.random.choice(idx, round(0.7*X.shape[0]), replace=False)
test_idx = np.setdiff1d(idx,train_idx)

xtrain = X[train_idx,:]
ytrain = a[1].numpy()[train_idx]

xtest = X[test_idx,:]
ytest = a[1].numpy()[test_idx]

dtrain = xgb.DMatrix(xtrain,label=ytrain)
dtest = xgb.DMatrix(xtest,label=ytest)

In [3]:
from hyperopt import fmin, tpe, hp, STATUS_OK, space_eval

best_val = 1.0

def objective(param):
    global best_val
    watchlist = [(dtrain,'train'),(dtest,'test')]
    ed1_results = dict()
    print(param)
    wbst = wxgb.train(param, dtrain, 100,watchlist,evals_result=ed1_results)
    
    if min(ed1_results['test']['error']) < best_val:
        print("NEW BEST VALUE!")
        best_val = min(ed1_results['test']['error'])
    
    return {'loss': min(ed1_results['test']['error']), 'status': STATUS_OK }

spc = {
    'btype': hp.choice('btype',['R','I','Rn','In']),
    'extra_dims': hp.choice('extra_dims',range(16)),
    'objective': hp.choice('objective',['binary:logistic']),
    'eval_metric':hp.choice('eval_metric',[['error']]),
    'eta': hp.loguniform('eta', -7, 0),
    'max_depth' : hp.choice('max_depth',range(1,11)),
    'subsample': hp.uniform('subsample', 0.5, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1),
    'colsample_bylevel': hp.uniform('colsample_bylevel', 0.5, 1),
    'min_child_weight': hp.loguniform('min_child_weight', -16, 5),
    'alpha': hp.choice('alpha', [0, hp.loguniform('alpha_positive', -16, 2)]),
    'lambda': hp.choice('lambda', [0, hp.loguniform('lambda_positive', -16, 2)]),
    'gamma': hp.choice('gamma', [0, hp.loguniform('gamma_positive', -16, 2)])
}

best = fmin(objective,
    space=spc,
    algo=tpe.suggest,
    max_evals=100)

{'alpha': 4.1826926934484426e-05, 'btype': 'In', 'colsample_bylevel': 0.6524141785664681, 'colsample_bytree': 0.7619674246849375, 'eta': 0.01730353148405771, 'eval_metric': ('error',), 'extra_dims': 9, 'gamma': 0, 'lambda': 0, 'max_depth': 3, 'min_child_weight': 0.12548050183681422, 'objective': 'binary:logistic', 'subsample': 0.6652741628856679}
Overwriting param `num_class`                          
Overwriting param `objective` while setting `obj` in train.
Taking first argument of eval_metric. Multiple evals not supported using xgboost backend.
Moving param `eval_metric` to an feval.                
Setting param `disable_default_eval_metric` to 1.      
[10:47:21] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:480: 
Parameters: { btype } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if yo

[4]	train-error:0.38865	test-error:0.36641                             

[5]	train-error:0.38865	test-error:0.36641                             

[6]	train-error:0.38865	test-error:0.36641                             

[7]	train-error:0.38865	test-error:0.36641                             

[8]	train-error:0.38865	test-error:0.36641                             

[9]	train-error:0.38865	test-error:0.36641                             

[10]	train-error:0.38865	test-error:0.36641                            

[11]	train-error:0.38865	test-error:0.36641                            

[12]	train-error:0.38865	test-error:0.36641                            

[13]	train-error:0.38865	test-error:0.36641                            

[14]	train-error:0.38865	test-error:0.36641                            

[15]	train-error:0.38865	test-error:0.36641                            

[16]	train-error:0.38865	test-error:0.36641                            

[17]	train-error:0.38865	test-error:0.36641        

[1]	train-error:0.20633	test-error:0.20356                             

[2]	train-error:0.20633	test-error:0.20356                             

[3]	train-error:0.20742	test-error:0.20356                             

[4]	train-error:0.20306	test-error:0.20356                             

[5]	train-error:0.20415	test-error:0.20356                             

[6]	train-error:0.20306	test-error:0.20356                             

[7]	train-error:0.20306	test-error:0.20356                             

[8]	train-error:0.20306	test-error:0.20356                             

[9]	train-error:0.20306	test-error:0.20356                             

[10]	train-error:0.20306	test-error:0.20356                            

[11]	train-error:0.20087	test-error:0.20356                            

[12]	train-error:0.20087	test-error:0.20102                            

[13]	train-error:0.20087	test-error:0.20102                            

[14]	train-error:0.19869	test-error:0.20102        

[0]	train-error:0.22052	test-error:0.21883                             

[1]	train-error:0.21616	test-error:0.19338                             

[2]	train-error:0.19323	test-error:0.18829                             

[3]	train-error:0.19323	test-error:0.19847                             

[4]	train-error:0.18887	test-error:0.18829                             

[5]	train-error:0.16812	test-error:0.19847                             

[6]	train-error:0.16921	test-error:0.17557                             

[7]	train-error:0.16485	test-error:0.19593                             

[8]	train-error:0.15284	test-error:0.18321                             

[9]	train-error:0.14629	test-error:0.19593                             

[10]	train-error:0.14629	test-error:0.18829                            

[11]	train-error:0.15066	test-error:0.19338                            

[12]	train-error:0.13865	test-error:0.19084                            

[13]	train-error:0.14520	test-error:0.19084        

[0]	train-error:0.22052	test-error:0.21883                             

[1]	train-error:0.22052	test-error:0.21883                             

[2]	train-error:0.22052	test-error:0.21883                             

[3]	train-error:0.21616	test-error:0.21629                             

[4]	train-error:0.21397	test-error:0.21629                             

[5]	train-error:0.21397	test-error:0.21374                             

[6]	train-error:0.21397	test-error:0.21374                             

[7]	train-error:0.21397	test-error:0.21374                             

[8]	train-error:0.21070	test-error:0.21374                             

[9]	train-error:0.20961	test-error:0.21374                             

[10]	train-error:0.21179	test-error:0.21120                            

[11]	train-error:0.21834	test-error:0.20356                            

[12]	train-error:0.21943	test-error:0.20611                            

[13]	train-error:0.21834	test-error:0.20102        

...

[10:52:29] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:480: 
Parameters: { btype } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-error:0.19214	test-error:0.19847                              

[1]	train-error:0.18777	test-error:0.19338                              

[2]	train-error:0.18122	test-error:0.19084                              

[3]	train-error:0.18013	test-error:0.19338                              

[4]	train-error:0.17358	test-error:0.20102                              

[5]	train-error:0.17031	test-error:0.19338                              

[6]	train-error:0.16157	test-error:0.18321                              

[7]	train-error:0.15721	test-error:0.17812                              

[8]	train-error:0.15611	test-error:0.18321           

Overwriting param `objective` while setting `obj` in train.             
Taking first argument of eval_metric. Multiple evals not supported using xgboost backend.
Moving param `eval_metric` to an feval.                                 
Setting param `disable_default_eval_metric` to 1.                       
[10:52:33] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:480: 
Parameters: { btype } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-error:0.12009	test-error:0.18066                              

[1]	train-error:0.11245	test-error:0.19084                              

[2]	train-error:0.11026	test-error:0.18829                              

[3]	train-error:0.09825	test-error:0.19084                              

[4]	train-error:0.09280	test-error:0.190

Overwriting param `num_class`                                           
Overwriting param `objective` while setting `obj` in train.             
Taking first argument of eval_metric. Multiple evals not supported using xgboost backend.
Moving param `eval_metric` to an feval.                                 
Setting param `disable_default_eval_metric` to 1.                       
[10:52:38] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:480: 
Parameters: { btype } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-error:0.10262	test-error:0.19084                              

[1]	train-error:0.09825	test-error:0.18066                              

[2]	train-error:0.09607	test-error:0.16794                              

[3]	train-error:0.09389	test-error:0.1730

{'alpha': 0.046198076483170654, 'btype': 'In', 'colsample_bylevel': 0.871051077473679, 'colsample_bytree': 0.5996928973381576, 'eta': 0.0017041802513983423, 'eval_metric': ('error',), 'extra_dims': 11, 'gamma': 1.7183207280158013, 'lambda': 0, 'max_depth': 1, 'min_child_weight': 6.791227733451724e-07, 'objective': 'binary:logistic', 'subsample': 0.6042218252547266}
Overwriting param `num_class`                                           
Overwriting param `objective` while setting `obj` in train.             
Taking first argument of eval_metric. Multiple evals not supported using xgboost backend.
Moving param `eval_metric` to an feval.                                 
Setting param `disable_default_eval_metric` to 1.                       
[10:52:42] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:480: 
Parameters: { btype } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some param

[95]	train-error:0.21397	test-error:0.21374                             

[96]	train-error:0.21397	test-error:0.21374                             

[97]	train-error:0.21397	test-error:0.21374                             

[98]	train-error:0.21397	test-error:0.21374                             

[99]	train-error:0.21397	test-error:0.21374                             

100%|██████████| 100/100 [05:24<00:00,  3.25s/trial, best loss: 0.160305]


In [5]:
print(best_val)
print(space_eval(spc, best))

0.160305
{'alpha': 0.012107447874868508, 'btype': 'R', 'colsample_bylevel': 0.992848768328804, 'colsample_bytree': 0.5864356970877662, 'eta': 0.0009592396424514969, 'eval_metric': ('error',), 'extra_dims': 13, 'gamma': 0, 'lambda': 0, 'max_depth': 6, 'min_child_weight': 1.4383188220215186e-07, 'objective': 'binary:logistic', 'subsample': 0.5530442561668147}
